In [3]:
library(limma)
library(reshape2)
library(Glimma)
library(ggplot2)
library(ggrepel)
# library(reshape)
library(edgeR)
library(gplots)
library(amap)
library(ggfortify)
library(data.table)
# library(Mus.musculus)


Attaching package: 'gplots'


The following object is masked from 'package:stats':

    lowess



Attaching package: 'data.table'


The following objects are masked from 'package:reshape2':

    dcast, melt




In [4]:

# change this
setwd("C:\\Users\\Pete\\DSC180B-PROJECT")

In [5]:
file_url = 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE44639&format=file'            
raw_dir = "data/raw/"
temp_dir = "data/temp/"
file_name = 'GSE44639_RAW.tar'

In [6]:
ingest_data <- function(file_url, file_name, raw_dir, temp_dir) {
    if (!dir.exists(raw_dir)) {
        dir.create(raw_dir, recursive = TRUE)
    }
    out = paste(raw_dir, file_name, sep="")
    utils::download.file(file_url, destfile=out, mode="wb")
    utils::untar(out, exdir=temp_dir)
}

ingest_data(file_url, file_name, raw_dir, temp_dir)

In [7]:
read_all_files_in_dir_with_columns <- function(file_dir, required_columns) {
    files <- list.files(path=file_dir)
    
    all <- list()
    
    for (i in files) {
        fp <- paste(file_dir, i, sep='')
        seqdata <- read.delim(gzfile(fp), stringsAsFactors = FALSE)
        columns <- colnames(seqdata)
        
        has_unidentified_col <- FALSE
        
        # ignore samples with columns not present in required_columns
        for (c in columns) {
            if (!any(required_columns==c)) {
                has_unidentified_col <- TRUE
                break
            }
        }
        
        if (has_unidentified_col) {
            next
        }
        
        sampleName = strsplit(fp, "/")[[1]][3]
        sampleName = strsplit(sampleName, ".txt")[[1]][1]
        
        
        cellType = strsplit(fp, "_")[1]
        cellType = cellType[[1]][3]
        cellType = strsplit(cellType, ".txt")
        cellType = cellType[[1]][1]
        
        healthy <- !grepl("P", fp, fixed=TRUE)
        
        # add healthy column
        seqdata$healthy <- healthy
        seqdata$cellType <- cellType
        seqdata$sampleName <- sampleName
        
        if (length(all) == 0) {
            all <- seqdata
        } else {
            all <- rbind(all, seqdata)
        }
    }
    
    return(all)
}

required_columns <- list("miRNA", "Chromosome", "Position", "Strand", "Total.miRNA.reads", "RPM..reads.per.million.")
all <- read_all_files_in_dir_with_columns(temp_dir, required_columns)

In [10]:
head(all)

,miRNA,Chromosome,Position,Strand,Total.miRNA.reads,RPM..reads.per.million.,healthy,cellType,sampleName
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<lgl>,<chr>,<chr>
1,hsa-miR-378c,chr10,132760851,-,7,1,TRUE,Naive,GSM1088200_M7_Naive
2,hsa-mir-1321,chrX,85090785,+,26,6,TRUE,Naive,GSM1088200_M7_Naive
3,hsa-miR-378d,chr8,94928250,-,2,1,TRUE,Naive,GSM1088200_M7_Naive
4,hsa-miR-2110,chr10,115933864,-,5,1,TRUE,Naive,GSM1088200_M7_Naive
5,hsa-mir-200b,chr1,1102484,+,2,1,TRUE,Naive,GSM1088200_M7_Naive
6,hsa-miR-125a-5p,chr19,52196507,+,7,1,TRUE,Naive,GSM1088200_M7_Naive


In [102]:
pivot_data <- function(countdata, value_column, groupNames, groupSizes) {
    ### pivot ingested miRNA counts data
    ### dataframe must have columns miRNA and sampleName
    # countdata - data with raw reads and from ingestion function
    # value_column - name of column with the reads
    # groupNames - list of group names
    # groupSizes - list of size of each group
    
    
    countdata <- dcast(countdata, miRNA ~ sampleName, value.var= value_column, fill= 0)
    rownames(countdata) <- countdata$miRNA
    countdata$miRNA <- NULL
    
    factoredGroups <- vector()
    for (i in 1:length(groupNames)) {
        curGroup <- rep(groupNames[i], groupSizes[i])
        factoredGroups <- c(factoredGroups, curGroup)
    }
    group <- factor(factoredGroups)
    toReturn <- list('countdata'=countdata, 'group'=group)
    return(toReturn)

}

filter_logcounts <- function(countdata, group) {
    countdata <- countdata + 0.5

    d <- DGEList(counts=countdata,group=group)

    # got values for filter from paper, Yuxia Zhang
    # filter all mirna with less than 1.5*(# of total samples) of counts across all samples. lowly expressing gene
    dd <- d[rowSums(d$counts) >= 1.5*ncol(d),]
    # multiply by million and divide by column library size to get RPM
    d<-1e+06 * dd$count/expandAsMatrix(d$samples$lib.size,dim(dd))
    # take the log RPM
    d<-log2(d)
    return(d)
}

In [110]:
# pret1 <- all[all$healthy == FALSE,] 
# naive <- all[all$cellType == 'Naive',] 
rTreg <- all[all$cellType == 'rTreg',] 

groupNames <- c("healthyrTreg", "pret1rTreg")
groupSizes <- c(8,7)
dataAndGroupList <- pivot_data(rTreg, "Total.miRNA.reads", groupNames, groupSizes)
df <- filter_logcounts(dataAndGroupList$countdata, dataAndGroupList$group)

Warning message in dcast(countdata, miRNA ~ sampleName, value.var = value_column, :
"The dcast generic in data.table has been passed a data.frame and will attempt to redirect to the reshape2::dcast; please note that reshape2 is deprecated, and this redirection is now deprecated as well. Please do this redirection yourself like reshape2::dcast(countdata). In the next version, this warning will become an error."


In [148]:
absF <- function(df, groupSize1, groupSize2) {
    # calculate absF, stat for between-group variability vs within-group. used for volcano plot
    F<-data.frame(nrow(df))
    # NAIVE
    x<-data.frame(nrow(df),2) # for calculating fold change later
    total_groups <- groupSize1+groupSize2
    for (i in 1:nrow(df)) {
        a<-groupSize1*abs(mean(df[i,1:groupSize1])-mean(df[i,1:total_groups]))+groupSize2*abs(mean(df[i,(groupSize1+1):total_groups])-mean(df[i,1:total_groups]))
        bb = 0
        for (j in 1:groupSize1) {
            b<-abs(df[i,j]-mean(df[i,1:groupSize1]))
            bb = bb +b
        }

        for (j in (groupSize1+1):total_groups) {
            b<-abs(df[i,j]-mean(df[i,(groupSize1+1):total_groups]))
            bb = bb +b
        }

        F[i]<-12*a/bb # 12 is scalar from the paper/ Yuxia Zhang


        # X contains the mean of the logRPM of each group.
        x[i,2]<-mean(df[i,1:groupSize1])
        x[i,1]<-mean(df[i,(groupSize1+1):total_groups])

    }
    names(F)<-rownames(df)
    rownames(x)<-rownames(df)
#     absFOver9.6<-length(F[,F>9.6]) # 9.6 chosen from paper. corresponds to less than 1% false discovery rate in limma package
    return(list(FVal_df=F, miRNA_means=x))
}
    


In [150]:
absFScores <- absF(df, 8, 7)
absFScores

$FVal_df
  hsa-let-7a hsa-let-7a-1 hsa-let-7a-2 hsa-let-7a-3 hsa-let-7a* hsa-let-7b
1   3.554506     1.250739    0.3868561     1.715975    3.281919   7.019244
  hsa-let-7b* hsa-let-7c hsa-let-7d hsa-let-7d* hsa-let-7e hsa-let-7f
1    4.543778   9.367175   2.976338   0.1341111   5.341475  0.2040721
  hsa-let-7f-1 hsa-let-7f-1* hsa-let-7f-2 hsa-let-7f-2* hsa-let-7g hsa-let-7g*
1     4.799709     0.4327971     2.625661       3.45382   3.254253    4.838311
  hsa-let-7i hsa-let-7i* hsa-miR-1 hsa-mir-1-1 hsa-mir-1-2 hsa-miR-100
1  0.6540079    6.675621  2.058289    2.527049    2.527049    3.473301
  hsa-miR-101 hsa-mir-101-1 hsa-mir-101-2 hsa-miR-101* hsa-miR-103a
1       4.108      2.835735     0.5218514     2.937521     6.033358
  hsa-mir-103a-1 hsa-mir-103a-2 hsa-miR-103b hsa-mir-103b-1 hsa-mir-103b-2
1       1.360091       1.517236     6.033358      0.7694986      0.7798478
  hsa-mir-106a hsa-miR-106a hsa-mir-106b hsa-miR-106b hsa-miR-106b* hsa-mir-107
1     1.168831     3.197785      8.48414     8.731821       1.93592   0.2335866
  hsa-miR-107 hsa-miR-10a hsa-miR-1179 hsa-mir-122 hsa-miR-122 hsa-miR-125a-3p
1    6.249815    2.008856     1.663008     3.42153    7.299579         3.88495
  hsa-miR-125a-5p hsa-miR-125b hsa-miR-126 hsa-miR-126* hsa-mir-1260
1       0.8474286     3.008011    1.003099     2.566482     4.779545
  hsa-miR-1260 hsa-mir-1260b hsa-miR-1260b hsa-mir-1268 hsa-miR-1268
1    0.2486858      2.207469     0.1562388    0.9637428     1.506648
  hsa-mir-1268b hsa-miR-1268b hsa-miR-127-3p hsa-miR-1270 hsa-miR-1271
1      2.505105      2.554388       5.253502     2.783013       1.2816
  hsa-mir-1273 hsa-mir-1273d hsa-mir-1273e hsa-mir-1273f hsa-mir-1273g
1     6.380786      4.480178      5.862868      5.317054      7.627593
  hsa-mir-1275 hsa-mir-1277 hsa-miR-1277 hsa-miR-128 hsa-mir-128-1
1     3.225665    0.7551646    0.6472818    4.514923      3.575566
  hsa-mir-128-2 hsa-miR-1285 hsa-mir-1285-1 hsa-miR-1287 hsa-miR-1292
1      4.560495    0.5486392       3.767555     2.261756    0.8751745
  hsa-mir-1301 hsa-miR-1301 hsa-miR-1304 hsa-mir-1306 hsa-mir-1307 hsa-miR-1307
1     2.792382     4.610274     2.544589        3.288     1.842595     7.190168
  hsa-mir-130a hsa-miR-130a hsa-mir-130b hsa-miR-130b hsa-mir-132 hsa-miR-132
1    0.5042878     4.708263     2.015053    0.7362441  0.01129048    3.124372
  hsa-mir-1321 hsa-miR-139-5p hsa-mir-140 hsa-miR-140-3p hsa-miR-140-5p
1    0.6951661       5.300953    6.816275       10.50558       4.627716
  hsa-mir-141 hsa-miR-141 hsa-mir-142 hsa-miR-142-3p hsa-miR-142-5p hsa-miR-143
1    2.087677   0.7406917    9.030003      0.5780237        4.96444     7.91623
  hsa-miR-144 hsa-miR-1468 hsa-mir-146a hsa-miR-146a hsa-mir-146b
1    2.492226     1.249162     4.631387     3.831618     1.327639
  hsa-miR-146b-3p hsa-miR-146b-5p hsa-miR-148a hsa-miR-148b hsa-mir-150
1        2.102893        9.500488     2.850673    0.6948252    3.962984
  hsa-miR-150 hsa-miR-150* hsa-mir-151 hsa-miR-151-3p hsa-miR-151-5p
1   0.2756844     7.381527    1.433491        5.16035       2.322175
  hsa-mir-151b hsa-miR-151b hsa-mir-152 hsa-miR-152 hsa-miR-153 hsa-mir-1537
1     2.284707     1.392737   0.1763374    1.618529    1.308786     2.605769
  hsa-mir-155 hsa-miR-155 hsa-mir-15a hsa-miR-15a hsa-miR-15a* hsa-mir-15b
1   0.4309636    2.911947    7.893853    11.16748      3.10043    3.137121
  hsa-miR-15b hsa-miR-15b* hsa-miR-16 hsa-mir-16-1 hsa-mir-16-2 hsa-miR-16-2*
1    8.373478     5.842964   5.413674     3.397216     3.275199      3.675605
  hsa-mir-17 hsa-miR-17 hsa-miR-17* hsa-miR-181a hsa-mir-181a-1 hsa-mir-181a-2
1  0.2731035   3.871882    9.291765     2.306507       1.424418       1.560772
  hsa-miR-181a-2* hsa-miR-181a* hsa-miR-181b hsa-mir-181b-1 hsa-mir-181b-2
1        4.703435      8.212811     9.090764      0.5821881       1.144048
  hsa-mir-181c hsa-miR-181c hsa-mir-181d hsa-miR-181d hsa-miR-182 hsa-mir-185
1     4.043259     2.996816       1.3728   0.07876076    2.103553   0.4180472
  hsa